In [1]:
%pip install faster-whisper
%pip install huggingface_hub
%pip install edge-tts
%pip install dspy
%pip install torch
%pip install sounddevice

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sounddevice as sd
import numpy as np
from faster_whisper import WhisperModel
from huggingface_hub import InferenceApi
from dspy import DynamicPrompt
import edge_tts
import asyncio

model = WhisperModel('small', device='cuda', compute_type='int8_float16')

inference = InferenceApi(repo_id='mistralai/Mistral-7B-Instruct', token='YOUR_HF_API_KEY')

def capture_audio(duration=5, fs=16000):
    print('Recording...')
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='float32')
    sd.wait() 
    return audio.flatten()

def apply_vad(audio, threshold=0.5):
    vad_audio = []
    for chunk in np.array_split(audio, len(audio) // int(0.02 * 16000)):
        if np.mean(np.abs(chunk)) > threshold:
            vad_audio.extend(chunk)
    return np.array(vad_audio)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\MS COMPUTER\anaconda3\envs\myenv\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\MS COMPUTER\anaconda3\envs\myenv\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\MS COMPUTER\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\MS COMPUTER\anaconda3\envs\myenv\lib\site-packages\traitlets\config\application.py", line 1075, in launch_in

ImportError: cannot import name 'DynamicPrompt' from 'dspy' (c:\Users\MS COMPUTER\anaconda3\envs\myenv\lib\site-packages\dspy\__init__.py)

In [2]:
def generate_response(prompt):
    response = inference(inputs=prompt, parameters={'max_length': 20})
    return response.get('generated_text', '')

def generate_dynamic_prompt(query):
    dp = DynamicPrompt()
    dp.add_prompt(f'Q: {query} A:', weight=1.0)
    dp.add_context('You are a helpful assistant.', weight=0.8)
    return dp.render()

In [3]:
async def text_to_speech(text, voice='en-US-JennyNeural', rate='+0%', pitch='+0%'):
    communicate = edge_tts.Communicate(text, voice=voice, rate=rate, pitch=pitch)
    await communicate.save('output_audio.mp3')

audio = capture_audio()
vad_audio = apply_vad(audio)
text_result = model.transcribe(vad_audio, vad_threshold=0.5)
transcribed_text = text_result['text']
print('Transcribed Text:', transcribed_text)

dynamic_prompt = generate_dynamic_prompt(transcribed_text)
response = generate_response(dynamic_prompt)
print('LLM Response:', response)

asyncio.run(text_to_speech(response, voice='en-US-GuyNeural', rate='+5%', pitch='-2%'))

NameError: name 'capture_audio' is not defined